In [1]:
# %load C60_jupyter_imports.py
%load_ext autoreload
%autoreload 2

import xarray as xr
import numpy as np
from dask.distributed import Client
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import xesmf as xe
from scipy.stats import linregress
import os
import scipy.signal as sps
import requests
# Lets import our functions. There are no tests here. Stored separately so as to not make a mess.
# Make sure in right dir first

os.chdir('/g/data/xv83/np1383/src_CAFE60_eqpac_analysis_code/')
from C60_obs_functions import cut_regrid_reynolds_sst,cut_process_sst_obs_trends,proc_landschutzer,process_co2_land_trends
from C60_helper_functions import lin_wrapper


from C60_helper_functions import solubility
from C60_helper_functions import convert_CAFEatm_CO2times
import PyCO2SYS as pyco2

import parcels
from parcels import FieldSet, ParticleSet, JITParticle,AdvectionRK4_3D,AdvectionRK4
import os
from datetime import timedelta

In [3]:
# Coefs / Inputs
depth_integration=100
lat=0
lon=270
ens=26
startyear=2000
endyear=2017

regrid_fratios=False
process_interim=False
process_trends=False

# Could do caco3 as well ...

In [2]:
# Set up the remote dask cluster. Can either use this version or a similar version above if building a LocalCluster.
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=2,memory="16GB")
client = Client(cluster)
cluster.scale(cores=32)
#cluster.adapt(minimum=2, maximum=16)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.161:42343,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [1]:
client

NameError: name 'client' is not defined

In [3]:
bgc=xr.open_zarr('/g/data/xv83/dcfp/CAFE60v1/ocean_bgc_month.zarr.zip')
bgc=bgc.rename({'xt_ocean':'lon','yt_ocean':'lat'})
bgc['lon']=bgc['lon']+360
eqpac=bgc.sel(lon=slice(120,290),lat=slice(-20,20))
eqpac['time']=eqpac.time.astype('datetime64[M]')
#eqpac.time
eqpac

,Array,Chunk
Bytes,214.49 GiB,18.30 MiB
Shape,"(720, 96, 50, 98, 170)","(1, 96, 3, 98, 170)"
Count,24481 Tasks,12240 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,214.49 GiB,18.30 MiB
Shape,"(720, 96, 50, 98, 170)","(1, 96, 3, 98, 170)"
Count,24481 Tasks,12240 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [6]:
# Load atmospheric PCO2 and calculate delta pCO2.
# Input files have been copied from:
# cd /scratch/v14/rxm599/enkf-bgc5/MOM/INPUT/
# cp co2_obs.nc /g/data/xv83/np1383/processed_data/co2_obs.nc
# cp bgc_param.nc /g/data/xv83/np1383/processed_data/bgc_params.nc

atm_co2=xr.open_dataset('../processed_data/co2_obs.nc',decode_times=False) #../co2_obs.nc # '/scratch/v14/rxm599/enkf-bgc5/MOM/INPUT/co2_obs.nc'
bgc_params=xr.open_dataset('../processed_data/bgc_params.nc',decode_times=False) #../co2_obs.nc # '/scratch/v14/rxm599/enkf-bgc5/MOM/INPUT/bgc_params.nc'

atm_co2=convert_CAFEatm_CO2times(atm_co2)
atm_co2=atm_co2.rename({'grid_x_T':'lon','grid_y_T':'lat'})
atm_co2['lon']=atm_co2['lon']+360
atm_co2_eq=atm_co2.sel(lon=slice(120,290),lat=slice(-20,20))
atm_co2_eq['lat']=eqpac.lat # Overwrite test

dpco2=eqpac.pco2.sel(ensemble=26).sel(time=slice('1960-01-01','2017-01-01'))-atm_co2_eq.co2.sel(time=slice('1960-01-01','2017-01-01')).load()
eqpac['dpco2']=dpco2 #Sea minus Air

In [4]:
# Load Physics
phys=xr.open_zarr('/g/data/xv83/dcfp/CAFE60v1/ocean_month.zarr.zip')
phys=phys.rename({'xu_ocean':'lon_x','yu_ocean':'lat_x'})
phys=phys.rename({'xt_ocean':'lon','yt_ocean':'lat'})

phys['lon']=phys['lon']+360
phys['lon_x']=phys['lon_x']+360

phys_eqpac=phys.sel(lon=slice(120,290),lat=slice(-20,20))
phys_eqpac

,Array,Chunk
Bytes,214.49 GiB,18.30 MiB
Shape,"(720, 96, 50, 98, 170)","(1, 96, 3, 98, 170)"
Count,24481 Tasks,12240 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,214.49 GiB,18.30 MiB
Shape,"(720, 96, 50, 98, 170)","(1, 96, 3, 98, 170)"
Count,24481 Tasks,12240 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [8]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.138:36109,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [9]:
#bgcdatvs=xr.open_dataset('../processed_data/model_proc_temp/epac_bgc.nc')
#physdatvs=xr.open_dataset('../processed_data/model_proc_temp/epac_phys.nc')

In [10]:
phys

,Array,Chunk
Bytes,1.36 TiB,118.65 MiB
Shape,"(720, 96, 50, 300, 360)","(1, 96, 3, 300, 360)"
Count,12241 Tasks,12240 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,1.36 TiB,118.65 MiB
Shape,"(720, 96, 50, 300, 360)","(1, 96, 3, 300, 360)"
Count,12241 Tasks,12240 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [6]:
phys_daily

,Array,Chunk
Bytes,633.62 GiB,39.55 MiB
Shape,"(16405, 96, 300, 360)","(1, 96, 300, 360)"
Count,16406 Tasks,16405 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,633.62 GiB,39.55 MiB
Shape,"(16405, 96, 300, 360)","(1, 96, 300, 360)"
Count,16406 Tasks,16405 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [9]:
# Load Physics
phys_daily=xr.open_zarr('/g/data/xv83/dcfp/CAFE60v1/ocean_month.zarr.zip')
phys_daily=phys_daily.rename({'xu_ocean':'lon_x','yu_ocean':'lat_x'})
phys_daily=phys_daily.rename({'xt_ocean':'lon','yt_ocean':'lat'})

phys_daily['lon']=phys_daily['lon']+360
phys_daily['lon_x']=phys_daily['lon_x']+360

phys_daily_eqpac=phys_daily.sel(lon=slice(120,290),lat=slice(-20,20))
phys_daily_eqpac

,Array,Chunk
Bytes,214.49 GiB,18.30 MiB
Shape,"(720, 96, 50, 98, 170)","(1, 96, 3, 98, 170)"
Count,24481 Tasks,12240 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,214.49 GiB,18.30 MiB
Shape,"(720, 96, 50, 98, 170)","(1, 96, 3, 98, 170)"
Count,24481 Tasks,12240 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


## Play with parcels quickly

In [10]:
import cftime
from datetime import timedelta, datetime

In [11]:
da=phys_daily_eqpac.sel(ensemble=26)
da['time']=da.time.indexes['time'].to_datetimeindex()

/local/v45/np1383/tmp/ipykernel_1849236/1831663178.py:2: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  da['time']=da.time.indexes['time'].to_datetimeindex()


In [12]:
variables={'U':'u',
           'V':'v',
           'W':'wt'}

dimensions={}
dimensions['U']={'lon':'lon_x','lat':'lat_x','depth':'st_ocean','time':'time'}#,'depth':'st_ocean'
dimensions['V']={'lon':'lon_x','lat':'lat_x','depth':'st_ocean','time':'time'}
dimensions['W']={'lon':'lon','lat':'lat','depth':'sw_ocean','time':'time'}#{'lat':'lat_x','lon':'lon_x','time':'time'}}

fieldset=FieldSet.from_xarray_dataset(da,variables,dimensions)

In [13]:
fieldset

In [14]:
da.u.shape

(720, 50, 300, 360)

In [15]:
pset = ParticleSet.from_line(fieldset=fieldset, pclass=JITParticle,
                             size=10,
                             start=(260,-1),
                             finish=(260,1),
                             time=datetime(2002, 1, 1))


In [ ]:
# https://tristansalles.github.io/EnviReef/6-addson/parcels.html

In [ ]:
output_nc = 'CurrentParticles.nc'
try:
    os.remove(output_nc)
except OSError:
    pass
output_file = pset.ParticleFile(name=output_nc, 
                                outputdt=timedelta(weeks=8))

pset.execute(AdvectionRK4_3D,
             runtime=timedelta(weeks=32),
             dt=timedelta(weeks=4),
             output_file=output_file)#recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})


sh: None: command not found
INFO: Compiled ArrayJITParticleAdvectionRK4_3D ==> /local/v45/np1383/tmp/parcels-13198/libc6e9660147a2464b3147bee706cef10b_0.so


In [ ]:
pset.show(field=fieldset.W, domain={'N':3, 'S':3, 'E':250 ,'W':260}, depth_level=2)

In [ ]:
output_file.export()

In [ ]:
parcels = xr.open_dataset(output_nc)
parcels

In [ ]:
pset

In [ ]:
depth_level=8

In [ ]:
fieldset.show(field=fieldset.W, domain={'N':20, 'S':20, 'E':15 ,'W':0}, depth_level=depth_level)